# Proyecto 03 - Sistemas de Recomendación


## Dataset: STEAM
## Consignas

**Recuerda descargar el dataset de [aquí](https://github.com/kang205/SASRec). Son dos archivos, uno de calificaciones y otro de información sobre los juegos.**

En este notebook te dejamos unas celdas para que puedas comenzar a trabajar con este dataset. Sin embargo, **deberás** modificarlas para hacer un mejor manejo de datos. Algunas cosas a las que deberás prestar atención (tal vez no a todas):
1. Tipos de datos: elige tipos de datos apropiados para cada columna.
2. Descartar columnas poco informativas.
3. Guardar en memoria datasets preprocesados para no tener que repetir código que tarde en correr.

### Exploración de datos

Dedícale un buen tiempo a hacer un Análisis Exploratorio de Datos. Elige preguntas que creas que puedas responder con este dataset. Por ejemplo, ¿cuáles son los juegos más populares?¿Y los menos populares?

### Filtro Colaborativo

Deberás implementar un sistema de recomendación colaborativo para este dataset. Ten en cuenta:

1. Haz todas las transformaciones de datos que consideres necesarias. Justifica.
1. Evalúa de forma apropiada sus resultados. Justifica la métrica elegida.
1. Elige un modelo benchmark y compara tus resultados con este modelo.
1. Optimiza los hiperparámetros de tu modelo.

Puedes implementar un filtro colaborativo a partir de la similitud coseno o índice de Jaccard. ¿Puedes utilizar los métodos de la librería Surprise? Si no es así, busca implementaciones (por ejemplo, nuevas librerías) que sean apropiadas.

Para comenzar a trabajar, puedes asumir que cada entrada es un enlace entre una persona usuaria y un item, **independientemente** de si la crítica es buena o mala. 

### Para pensar, investigar y, opcionalmente, implementar
1. ¿Cómo harías para ponerle un valor a la calificación?
1. ¿Cómo harías para agregar contenido? Por ejemplo, cuentas con el género, precio, fecha de lanzamiento y más información de los juegos.
1. ¿Hay algo que te gustaría investigar o probar?

## Parte A - Exploración de Datos

### 1. CONVERTIMOS ARCHIVOS JSON A CSV

In [1]:
import gzip
import pandas as pd

def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

#### 1.1 REVIEWS

In [ ]:
contador = 0
data_reviews = []
 # Vamos a guardar una de cada 10 reviews para no llenar la memoria RAM.
n = 10
for l in parse('steam_reviews.json.gz'):
    if contador%n == 0:
        data_reviews.append(l)
    else:
        pass
    contador += 1

In [ ]:
data_reviews = pd.DataFrame(data_reviews)

In [ ]:
data_reviews.head()

In [ ]:
data_reviews.to_csv('new_data_reviews.csv')

#### 1.2 GAMES

In [ ]:
data_games = []
for l in parse('steam_games.json.gz'):
    data_games.append(l)
data_games = pd.DataFrame(data_games)

In [ ]:
data_games.head()

In [ ]:
data_games.to_csv('new_data_games.csv')

### 2. BREVE DESCRIPCIÓN DE STEAM

__Steam__ es un sistema de distribución de juegos multiplataforma en línea, con alrededor de 75 millones de usuarios activos, alrededor de 172 millones de cuentas en total, que aloja más de 3000 juegos, lo que lo convierte en una plataforma ideal para el tipo de trabajo que aquí se presenta. El conjunto de datos contiene registros de más de 3200 juegos y aplicaciones.  

Steam es un servicio de distribución digital de videojuegos de Valve. Se lanzó como un cliente de software independiente en septiembre de 2003 como una forma de que Valve proporcionara actualizaciones automáticas para sus juegos y luego, se expandió para incluir juegos de editores externos. Steam también se ha expandido a una tienda digital móvil y basada en la web en línea.  

De acuerdo con la **popularidad del juego, la similitud de la descripción del juego, la calidad del juego y la preferencia del jugador por el juego**, recomiendan el juego correspondiente al jugador del juego, de modo que Steam obtenga un mayor grado de satisfacción del cliente.

### 3. ANÁLISIS EXPLORATORIO DE DATOS

1. __Se importan las librerías__ necesarias para trabajar en la consigna.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import pandas as pd

import gc # garbage collector

from surprise import Dataset     
from surprise import Reader
from surprise.model_selection import train_test_split

2. __Se realiza la carga el dataset__ usando las funcionalidades de Pandas.

__DATA REVIEW__

In [ ]:
new_data_reviews = pd.read_csv('new_data_reviews.csv')

In [ ]:
new_data_reviews.shape # Filas y columnas

* *El Dataset, cuenta con **779.307 Filas**, y **13 Columnas**.*

In [ ]:
new_data_reviews.head(3) # Primeras 3 instancias (filas)

__DATA GAMES__

In [ ]:
new_data_games = pd.read_csv('new_data_games.csv')

In [ ]:
new_data_games.shape # Filas y columnas

* *El Dataset, cuenta con **32.135 Filas**, y **17 Columnas**.*

In [ ]:
new_data_games.head(3) # Primeras 3 instancias (filas)

3. __Valores Faltantes:__ se imprimen en pantalla los nombres de las columnas y cuántos valores faltantes hay por columna. En un principio es a mera exposición, ya que por el momento no vamos a descartar ninguno de ellos, ni realizar imputación de datos.

__DATA REVIEW__

In [ ]:
new_data_reviews.isnull().sum() # Nombres de las columnas y su cantidad de faltantes

* *Variables con elementos faltantes:*  
    *1. `compensation` **98%** (764.719);*  
    *2. `found_funny` **86%** (659.143);*  
    *3. `user_id` 59% c/u (461.967);*  
    *4. `hours` 0,3% (2.637);*  
    *5. `text` 0,2% (1.839);*  
    *6. `product` 0,2% (1.566).*

__DATA GAMES__

In [ ]:
new_data_games.isnull().sum() # Nombres de las columnas y su cantidad de faltantes

* *Casi todas las Variables tienen elementos faltantes. Detallamos las principales:*  
    *1. `discount_price` **98%** (31.910);*  
    *2. `metascore` **98%** (29.528);*  
    *3. `publisher` 59% c/u (8.062);*  
    *4. `sentiment` 0,3% (7.182);*  
    *5. `developer` 0,2% (3.299);*  
    *6. `genres` 0,2% (3.283).*
    
* *Cabe aclarar que la columna `id`, sólo tiene 2 valores faltantes.*
* *`metascore` refiere a la media de todas las reseñas recibidas para dicho juego.*

4. Reseñas a partir de la calificación __sentiment.__

In [ ]:
pd.unique(new_data_games['sentiment'])

In [ ]:
print(new_data_games['sentiment'].value_counts())

In [ ]:
print(new_data_games['sentiment'].value_counts().sum())

* *En total tenemos 24953 reseñas a partir de `sentiment`.*

In [ ]:
sns.countplot(data = new_data_games, y = 'sentiment', order = new_data_games['sentiment'].value_counts().index, palette='pastel')
plt.title('Número de Calificaciones por Tipo')

* *Podemos observar, que existe un tipo de calificación -`sentiment`-, que divide a los juegos en distintas reseñas, como ser, Muy Positivo, Positivo, Negativo, Muy Negativo, etc..*
* *Sin embargo, también tenemos dentro de la misma, reseñas que van de 1 user reviews a 9 user reviews, las cuales son poco viables rankear, ya que es difícil dar un orden a las mismas.*

5. Reseñas a partir de la calificaciones __metascore.__

In [ ]:
pd.unique(new_data_games['metascore'])

In [ ]:
print(new_data_games['metascore'].value_counts())

In [ ]:
print(new_data_games['metascore'].value_counts().sum())

* *En total tenemos 2607 reseñas a partir de la calificación `metascore`.*

In [ ]:
plt.figure(figsize = (15,13))
sns.countplot(data = new_data_games, y = 'metascore', order = new_data_games['metascore'].value_counts().index, palette='pastel')
plt.title('Número de Calificaciones por Tipo')

* *Si bien `metascore` parece ser una buena forma de darle puntuación a los juegos, la cantidad de calificaciones disponibles es realmente baja en función al dataset total de `reviews`.*
* *En total tenemos 2607 reseñas a partir de `metascore`, lo cual representa un 0,3% del dataset.*

### 4. PREPARACIÓN Y TRANSFORMACIÓN DE DATOS PARA RECOMENDACIÓN COLABORATIVA

* Los métodos de **filtrado colaborativo** construyen un modelo basado en el comportamiento pasado de los usuarios (artículos comprados anteriormente, películas vistas y calificadas, etc.) y utilizan las decisiones tomadas por los usuarios actuales y otros. Este modelo se utiliza luego para predecir elementos (o calificaciones de elementos) en los que el usuario puede estar interesado.
    * Ventajas: no necesitamos tener información acerca de los productos.
    * Desventajas: necesitamos tener la matríz de utilidad (que es muy dispersa) y llenarla es costosa en tiempo y dinero.
  
  
* Para implementarlo, necesitamos un dataset donde cada fila represente un `usuario`, un `juego` y la `calificación del usuario` a ese juego. Es decir, tiras de tres componentes. Hay otra información que puede ser útil, pero con esos tres datos ya podemos implementar un filtro colaborativo.

CASO PARTICULAR STEAM
* No hay registros tanto en el sitio web Steam, sobre las calificaciones continuas de estos usuarios. En realidad, en la plataforma, los usuarios sólo dan "Recomendación" o "No Recomendación", lo que significa revisiones binarias, positivas y negativas, incluso en el sitio web del usuario, todavía no hay ningún mecanismo sobre las calificaciones continuas como una estrella a cinco estrellas.
* Para obtener calificaciones continuas sobre la interacción entre los usuarios y los juegos, debemos suponer un mecanismo de interacción de calificación de los juegos por parte de los usuarios. Ya que las concentraciones de los usuarios sobre los juegos pueden ser ajustadas por sus `tiempos de juego`, podemos asumir que el tiempo de juego es una información bastante persuasiva sobre los intereses de los usuarios.
* Por lo tanto, __aquí asumimos que el `tiempo de juego` es una parte muy importante de los intereses.__

#### 4.1 DATA REVIEWS

1. Seleccionamos aquellos **features que nos seran útiles** a la hora de realizar las predicciones.

* *Según lo explicado en el punto anterior, en éste caso vamos a considerar los features de `username`, `product_id` y `hours`, ya que son los que nos van a ser útiles a la hora de realizar nuestro filtro colaborativo*.

In [ ]:
df = pd.read_csv('new_data_reviews.csv', dtype={'hours': np.float, 'product_id': np.int})
print(df.shape)

In [ ]:
df1 = df[['username','hours','product_id']]
df1

* *Nos quedamos con los 3 features indicados anteriormente.*

2. __Valores Faltantes:__ visualización y tratamiento.

In [ ]:
df1.isnull().sum()

* *Los valores fatantes representan menos del 0,3% del total de instancias, por lo que se procede a eliminarlos, ya que no deberían generar grandes distorsiones en el dataset.*

In [ ]:
df2 = df1.dropna()

In [ ]:
df2.isnull().sum()

In [ ]:
print(df2.shape)

3. __Outliers:__ visualización y tratamiento de valores atípicos en `horas`, la cual será la que utilizaremos luego para elaborar los Ratings.

In [ ]:
plt.figure(figsize = (6,4))
sns.boxplot(data = df2, y = 'hours', palette= 'pastel')
plt.title('Cantidad Horas Jugadas por Usuario por Juego')
plt.ticklabel_format(axis = 'y', style = 'plain')

* *Se procede a descartar los datos atípicos para `horas`, en éste caso aquellos valores ubicados por encima de 21.000 horas de juego.*

In [ ]:
mask_hours = (df2['hours'] <= 21000)
df3 = df2[mask_hours]

df3['hours'].describe()

In [ ]:
print(df3.shape)

* *El **Dataset Final de Reviews con el que vamos a trabajar**, representa aprox. un **99,7% del Dataset Original Descargado**.*

4. __Encoders:__ aplicación de LabelEncoder s/ `username`.

* *Cada nombre de usuario es **único**, lo cual se refleja en el feature `username` de nuestro dataset.*
* *Si bien Surpr!se puede trabajar con features bajo éstas condiciones, **se decide asignarle un Id a cada usuario único**, a fin de facilitar comparaciones a futuro.*

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df3['username'] = le.fit_transform(df3['username'])
print(df1['username'])

In [ ]:
df3

* *Como se observa en la columna de `username`, ahora cada usuario se encuentra representado por un Id Number.*

5. __Determinación de Calificaciones:__ confección de un `rating` a partir de las horas jugadas.

* *La **cantidad de horas jugadas por cada usuario para cada juego**, será para el presente estudio, determinante a la hora de establecer las calificaciones.*
* *Se establecerán 5 puntuaciones que **van del 1 a 5**, determinadas en función a la distribución de los datos (quintiles).*
* *Se escoje dicha elección, para que en cada una de las puntuaciones se halle una cantidad de valores similares.*

* *A modo de demostración, se expone la concentración de datos resultante para un tipo de rating, si simplemente hubiéramos optado por tomar el valor máximo de horas, es decir 20573, y lo dividiéramos en 5 Bins.*

In [ ]:
demo = df2

In [ ]:
bins = [0, 4114.6, 8229.2, 12343.8, 16458.4, 20573]
labels =[1,2,3,4,5]

demo['rating'] = pd.cut(demo['hours'], bins,labels=labels)

In [ ]:
demo['rating'].value_counts(normalize=True)

In [ ]:
fig = plt.figure()
fig, ax = plt.subplots(figsize = (6,4))
plt.title('Cantidad de Calificaciones por Rating')
sns.countplot(data=demo, x ='rating')

* *Vemos que visualmente, todas las calificaciones prarecerían quedar en **rating = 1**, y ésto ocurre porque concentra el **99,85%** de los datos.*

* *Ahora sí, **aplicamos la modalidad elegida** de dividir los datos en **quintiles**.*

In [ ]:
df3['rating'] = pd.qcut(df3.hours, 5, labels=['1', '2', '3', '4', '5'])
print (df3)

In [ ]:
df3['rating'].value_counts(normalize=True)

* *Se cumple la simétrica distribución de los datos.*
* *A continuación graficamos.*

In [ ]:
fig = plt.figure()
fig, ax = plt.subplots(figsize = (6,4))
plt.title('Cantidad de Calificaciones por Rating')
sns.countplot(data=df3, x ='rating')

In [ ]:
pd.unique(df3['rating'])

* *Los valores resultan categóricos, ordenados de menor a mayor.*
* *Los pasamos a enteros, con el fin de poder seguir explorando sus datos.*

In [ ]:
df3['rating'] = df3['rating'].astype(int)

* *Eliminamos la columna de `hours`, dejando en su remplazo la confeccionada de `rating` y reordenamos para una mejor visualización.*

In [ ]:
final_reviews = df3[['username','product_id','rating']]
final_reviews

In [ ]:
final_reviews.dtypes

* *Tenemos int32 para todos los features, lo cual nos permitirá un mejor procesamiento de datos en función a la memoria a utilizar.*

* *Obtenemos finalmente el **Dataset de Reviews**, con Usuarios, Id de Productos y Rating, que vamos a utilizar para seguir explorando los datos y sus relaciones con Data Games y además, será la base para entrenar el modelo elegido y realizar las recomendaciones de juegos propuestas.*

In [ ]:
if True:
    final_reviews.to_csv('final_reviews.csv', index= False) # Guardamos el Dataset modificado en un nuevo archivo

#### 4.2 DATA GAMES

1. Seleccionamos aquellos **features que nos seran útiles** a la hora de realizar las predicciones.

* *En éste caso, serán útiles las columnas de `title`, para poder identificar el nombre de los juegos, y del `id` de los juegos, para realizar el cruce de datos con Data Reviews, ya que es éste último el feature, el que tienen en común ambos Datasets.*

In [ ]:
df_titulo = pd.read_csv('new_data_games.csv', encoding = "ISO-8859-1", usecols = [4,13])
print(df_titulo.shape)
df_titulo.head()

2. Se **intercambian** las columnas, y se **renombra** la de `id`, a fin de que coincida con el dataset de Reviews.

In [ ]:
df_titulo = df_titulo[['id','title']]
df_titulo.head()

In [ ]:
df_new = df_titulo.rename(columns={'id':'product_id'})
df_new

3. __Valores Faltantes:__ visualización y tratamiento.

In [ ]:
df_new.isnull().sum()

* *Los valores fatantes representan el 6,4% del total de instancias.*
* *Se procede a eliminar todos ellos, ya que sin `product_id` no podemos cruzar los datos con el dataframe de Reviews, y sin `title` no podremos realizar las recomendaciones.*

In [ ]:
df_new_2 = df_new.dropna()

In [ ]:
df_new_2

In [ ]:
df_new_2.isnull().sum()

* *El **Dataset Final de Games con el que vamos a trabajar**, representa aprox. un **93,6% del Dataset Original Descargado**.*

4. Tratamiento del feature `product_id`, a fin de **indexarla** para realizar el cruce con el Dataset de Reviews en Surpr!se.

In [ ]:
df_new_2.dtypes

* *Precisamos que el tipo de dato sea entero para `product_id`, a fin de poder realizar su indexación.* 

In [ ]:
df_new_2[('product_id')] = df_new_2['product_id'].astype(int)

* *Ahora sí, se indexa la columna `pruduct_id`.*

In [ ]:
df_title = df_new_2.set_index('product_id', drop=True)
df_title

In [ ]:
df_title.dtypes

* *Luego, el tipo de dato de `pruduct_id` y `title` resultan ser **object**, siendo acorde para llevar a cabo nuestras recomendaciones a posteriori.*

5. __Eliminación__ de juegos __repetidos__.

* *Se eliminan aquellos valores que se encuentran duplicados en `product_id`, ya que sólo puede haber 1 juego con el mismo Id.*

In [ ]:
print(df_title.loc[612880])

In [ ]:
df_title = df_title[~df_title.index.duplicated(keep='first')]

In [ ]:
df_title

* *Sólo había 1 juego duplicado.*

* *Obtuvimos el **Dataset de Games** final, con Usuarios y Id de Productos como Index, para poder realizar el cruce de datos con Reviews, y que será utilizado para los mismos objetivos antes nombrados, es decir, seguir explorando los datos, y servir de base para entrenar el modelo elegido y realizar las recomendaciones de juegos propuestas.*

### 5. EXPLORANDO EL COMPORTAMIENTO DE LOS DATOS Y SU RELACIÓN ENTRE AMBOS DATASETS

* *Realizamos distintas preguntas, con el fin de realizar un Análisis Exploratorio de Datos más profundo y enfocado a nuestro objetivo de Recomendar Juegos.*

1. ¿Cuántos usuarios únicos hay?

In [ ]:
print(len(final_reviews['username'].unique()))

* *539030 usuarios calificaron juegos.*

2. ¿Cuántos juegos únicos hay?

In [ ]:
print(len(df_new_2['product_id'].unique())) # Se utiliza df_new_2, porque es donde product_id aún no se encuentra indexado

* *En total se trabajará con 30083 juegos.*

3. ¿Cuántas reseñas se realizaron?

In [ ]:
final_reviews['rating'].shape

* *Existen un total de 776650 calificaciones realizadas.*

4. Podemos obtener el nombre de un juego dado su `Id`.

In [ ]:
product_id = 4574
print(df_new_2.loc[product_id])

In [ ]:
product_id = 27432
print(df_new_2.loc[product_id])

5. ¿Cuántos juegos calificó cada usuario?

In [ ]:
calificaciones_por_usuario = final_reviews.username.value_counts()
calificaciones_por_usuario

* *La primer columna es el ID del usuario y la segunda, la cantidad de calificaciones que dió.*
* *El usuario 5442 es el que mas calificaciones realizó (juegos jugó), con un total de 213 calificaciones.*

6. ¿Cómo es la distribución del número de calificaciones por usuario?

In [ ]:
calificaciones_por_usuario.hist(log = True)

plt.xlabel('Cantidad de Calificaciones')
plt.ylabel('Cantidad de Usuarios')
plt.title('Cantidad de Calificaciones por Usuarios')
plt.show()

* *Vemos cuántas calificaciones de juegos realizó cada usuario.*
* *Alrededor del 10% de los usuarios, son los que más calificaciones han realizado (o juegos han jugado).*

7. ¿Cuáles son los juegos más populares? ¿Cuántas calificaciones tiene? ¿Y los juegos menos populares?

In [ ]:
juegos_por_jugados = final_reviews.product_id.value_counts()
juegos_por_jugados.index = df_title.loc[juegos_por_jugados.index].title
juegos_por_jugados

* *Se realiza un conteo por juego, de los valores únicos por juego (para c/ Id de juego, cuántos hay).*
* *El juego jugado por mayor cantidad de usuarios, es Team Fortress con 18372 calificaciones.*
* *5 de los juegos jugados 1 sola vez (menos calificados) son The Perks of Being a Wallflower, CitiesCorp Concept - Build Everything on Your Own, Island Racer, Island Racer, The Frost y DP Animation Maker.*

8. ¿Cuál es la calificación promedio de cada juego?

* *Primero, unimos los dataset de reviews y games*

In [ ]:
game_data = pd.merge(final_reviews, df_title, on='product_id')
game_data

* *Observamos los primeros 5 juegos mejor puntuados, en forma descendente.*

In [ ]:
game_data.groupby('title')['rating'].mean().sort_values(ascending=False).head()

* *Sin embargo, hay un problema. Un juego puede llegar a la cima de la lista anterior incluso si sólo un usuario le ha dado cinco estrellas. Por lo tanto, las estadísticas anteriores pueden ser engañosas. Normalmente, un juego que es realmente bueno obtiene una calificación más alta por un gran número de usuarios.*
* *Ahora volvamos a ver el número total de calificaciones de juego:*

In [ ]:
game_data.groupby('title')['rating'].count().sort_values(ascending=False).head()

* *Ahora que sabemos que tanto la calificación promedio por juego como el número de calificaciones por juego son atributos importantes, crearemos un nuevo marco de datos que contenga ambos atributos.*
* *Crearemos un nuevo dataframe de datos llamado `ratings_mean_count` y primero añadiremos la clasificación media de cada juego a este dataframe de datos de la siguiente manera:*

In [ ]:
ratings_mean_count = pd.DataFrame(game_data.groupby('title')['rating'].mean())

* *A continuación, añadiremos el número de calificaciones de un juego al cuadro de datos, con el conteo de la media de calificaciones.*

In [ ]:
ratings_mean_count['rating_counts'] = pd.DataFrame(game_data.groupby('title')['rating'].count())
ratings_mean_count.head()

* *Podemos ver el título del juego, junto con la calificación promedio y el número de calificaciones de los juegos.*

In [ ]:
plt.figure(figsize=(7,5))
plt.rcParams['patch.force_edgecolor'] = True
ratings_mean_count['rating'].hist(bins=50)
plt.xlabel('Calificación Promedio')
plt.ylabel('Cantidad de Juegos')
plt.title('Cantidad de Juegos por Calificación Promedio')

* *En nuestro caso, los juegos con un mayor número de valoraciones suelen tener también una valoración media baja.*

## Parte B - Modelo de Machine Learning

Los sistemas de recomendación se encuentran entre las aplicaciones más populares de la ciencia de datos en la actualidad. Se utilizan para predecir la "calificación" o "preferencia" que un usuario le daría a un artículo. Casi todas las grandes empresas de tecnología los han aplicado de alguna forma. Amazon lo usa para sugerir productos a los clientes, YouTube lo usa para decidir qué video reproducir a continuación en la reproducción automática y Facebook lo usa para recomendar páginas que les gusten y personas a seguir.

__Motores de filtrado colaborativo:__ estos sistemas se utilizan ampliamente e intentan predecir la calificación o preferencia que un usuario daría a un elemento en función de las calificaciones y preferencias pasadas de otros usuarios. Los filtros colaborativos no requieren metadatos de elementos como sus homólogos basados en contenido.  

Hay __3 enfoques:__  
* Filtrado colaborativo usuario-usuario;
* Filtrado colaborativo item-item y;
* Factorización matricial.

*Se trabajará con __Surpr!se__, tanto para el Benchmark como en el modelo SVD elegido, para construir y analizar nuestro sistemas de recomendación, trabajando para ello con datos de calificación explícitos.*

### 1. MODELO BENCHMARK: FILTRO COLABORATIVO ITEM-ITEM + KNN BASIC

#### 1.1 ENCONTRANDO SIMILITUDES ENTRE JUEGOS

__FILTRADO COLABORATIVO ITEM-ITEM (JUEGO-JUEGO)__

Ventajas:
* La recomendación no necesita entrenarse con frecuencia a pesar de que cambien las preferencias de los usuarios.
* Es computacionalmente más barato, ya que, en muchos casos, hay muchos más usuarios que elementos. Tiene sentido utilizar el filtrado basado en elementos en este caso.

Un ejemplo famoso de filtrado basado en elementos es  el  motor de recomendaciones de Amazon .

* *En el presente análisis, usaremos la **correlación entre las clasificaciones** de un juego como la **métrica de la similitud**.*

* *Utilizaremos el dataset fusionado en el punto anterior **(game_data)**, ya que tiene los features seleccionados de los datasets de reviews y games en uno solo.*
* *Vamos a descartar usuarios con el objetivo de achicar la base de datos. Se realizará de una manera **ad-hoc**.*
* *Descartaremos aquellos usuarios que califican poco (menos de 5 calificaciones) o mucho (más de 5000 calificaciones).*

In [ ]:
mask_usuarios_descartables = np.logical_or(game_data.username.value_counts() <= 5, game_data.username.value_counts() > 5000)
usuarios_descartables = mask_usuarios_descartables[mask_usuarios_descartables].index.values
print(len(usuarios_descartables))

In [ ]:
mascara_descartables = game_data.username.isin(usuarios_descartables)
print(mascara_descartables.sum())

In [ ]:
print(game_data.shape)
game_data = game_data[~mascara_descartables]
print(game_data.shape)

* *También vamos a descartar también aquellos juegos que tengan pocas calificaciones (menos de 100). Esto, lo hacemos con el objetivo de achicar la matriz de utilidad aún más.*

In [ ]:
mask_items_descartables = game_data.product_id.value_counts() <= 100
# mask_items_descartables
items_descartables = mask_items_descartables[mask_items_descartables].index.values
# items_descartables
print(len(items_descartables))

In [ ]:
mascara_descartables = game_data.product_id.isin(items_descartables)
print(mascara_descartables.sum())

In [ ]:
print(game_data.shape)
metascore_data = game_data[~mascara_descartables]
print(game_data.shape)

* *Para encontrar la **correlación entre las clasificaciones del juego**, necesitamos crear una matriz donde cada columna sea el nombre del juego y cada fila contenga la clasificación asignada por un usuario específico a ese juego.*
* *Esta matriz tendrá **muchos valores nulos**, ya que cada juego no está clasificado por todos los usuarios.*
* *Crearemos la matriz de títulos de juegos y las correspondientes clasificaciones de los usuarios.*

In [ ]:
user_game_rating = game_data.pivot_table(index='username', columns='title', values='rating')
user_game_rating

* *Vemos que nuestra matriz, posee 8898 filas y 8611 columnas, por lo que podremos trabajar con más facilidad.*
* *Cada columna contiene todas las clasificaciones de los usuarios de un juego en particular.*

* *Primero, chequeamos cuales son los juegos más populares luego de los filtros.*

In [ ]:
game_data.groupby('title')['rating'].count().sort_values(ascending=False).head()

* *`Team Fortress 2` sigue siendo el que más calificaciones tiene, mientras que `Rust` conserva el segundo lugar.*

* *Luego, buscaremos todas las clasificaciones de usuarios para el juego `Team Fortress 2` y encontremos los juegos similares a él.*
* *Escogimos éste juego porque, como se indicó anteriormente, tiene el mayor número de clasificaciones y queremos encontrar la correlación entre los juegos que tienen un mayor número de clasificaciones.*

In [ ]:
team_fortress_2_ratings = user_game_rating['Team Fortress 2']
team_fortress_2_ratings.head()

* *Recuperaremos todas los juegos que son similares a `Team Fortress 2`.*
* *Podemos encontrar la correlación entre las clasificaciones de usuario de Team Fortress 2 y todas los demás juegos usando la función corrwith() como se muestra a continuación:*

In [ ]:
games_like_team_fortress_2 = user_game_rating.corrwith(team_fortress_2_ratings)

corr_team_fortress_2 = pd.DataFrame(games_like_team_fortress_2, columns=['Correlation'])
corr_team_fortress_2.dropna(inplace=True)
corr_team_fortress_2.head()

* *Ahora, vamos a ordenar los juegos en orden descendente de correlación para ver los juegos altamente correlacionadas en la parte superior.*

In [ ]:
corr_team_fortress_2.sort_values('Correlation', ascending=False).head()

* *Podemos ver que algunos juegos que tienen una alta correlación con Team Fortress 2 no son muy conocidas.*
* *Esto muestra que la correlación por sí sola no es una buena medida para la similitud porque puede haber un usuario que haya visto Team Fortress 2 y otros pocos juegos y que los haya calificado a todos con 5.*
* *Una solución a este problema es recuperar sólo aquellos juegos correlacionados que tengan al menos más de 50 clasificaciones.*
* *Para ello, añadiremos la columna rating_counts del cuadro de datos rating_mean_count a nuestro cuadro de datos corr_team_fortress_2.*

* *Primero veamos la calificación promedio de cada juego.*

In [ ]:
ratings_mean_count_ = pd.DataFrame(game_data.groupby('title')['rating'].mean())

* *Luego, incorporamos el número de clasificaciones de un juego al cuadro de datos de la cuenta media de clasificaciones.*

In [ ]:
ratings_mean_count_['rating_counts'] = pd.DataFrame(game_data.groupby('title')['rating'].count())
ratings_mean_count_.head()

In [ ]:
corr_team_fortress_2 = corr_team_fortress_2.join(ratings_mean_count_['rating_counts'])
corr_team_fortress_2.head()

* *Ahora, filtremos los juegos correlacionados con Team Fortress 2, que tienen más de 50 clasificaciones.*

In [ ]:
corr_team_fortress_2[corr_team_fortress_2 ['rating_counts']>50].sort_values('Correlation', ascending=False).head()

* *Podemos ver los juegos que están altamente correlacionadas con Team Fortress 2.*
* *Los juegos de la lista son todos juegos disparos en primera persona, y como Team Fortress 2 es un juego muy famoso en ese género, hay una alta probabilidad de que estos juegos estén altamente correlacionadas.*
* *Por lo tanto, **hemos creado un simple sistema de recomendación**.*

#### 1.2. MODELO BENCHMARK: SURPRISE - KNN BASIC

* *Se elige como Benckmark, un simple algoritmo de filtrado colaborativo basado en memoria, que se deriva directamente de un enfoque básico de vecinos más cercanos.*

In [ ]:
reader = Reader()

from surprise import KNNBasic
from surprise import Dataset
from surprise import accuracy

data = Dataset.load_from_df(game_data[['username','product_id','rating']], reader)

trainset, testset = train_test_split(data, test_size=.25)

# Se construye el algoritmo y se entrena.
algo_KNN = KNNBasic()
algo_KNN.fit(trainset)
predictions_KNN = algo_KNN.test(testset)
accuracy.rmse(predictions_KNN)

* *Si bien el error obtenido medido a través de RMSE resulta razonable, esperamos obtener un mejor resultado con el modelo elegido SVD, que desarrollaremos en el siguiente apartado.*

### 2. MODELO PREDICTIVO ELEGIDO: RECOMENDACIÓN COLABORATIVA + MODELO SVD

#### 2.1 MODELO PREDICTIVO ELEGIDO: FACTORIZACIÓN MATRICIAL. MODELO SVD CON SURPR!SE

* *El **Modelo de ML elegido es SVD**, basado en factorización matricial.*
* *El **Dataset utilizado** para llevar a cabo el modelo, es el de **final_reviews**, que luego será cruzado con df_title con el fin de visualizar el título de los juegos recomendados, una vez realizadas las predicciones:* 

* *Llevaremos adelante los siguientes pasos:*
    * *Se carga el Dataset.*
    * *Ya aplicamos anteriormente el Reader, para que Surpr!se pueda leer el dataset.*
    * *Se crea el Dataset de Surpr!se usando `Dataset.load_from_df`.*
    * *Se realiza un train_test_split.*
    * *Se entrena un algoritmo SVD.*
    * *Entrenamos sobre el `trainset`.*
    * *Predecimos sobre el `testset`.*
    * *Para el conjunto de `testset`, evaluamos el error RMSE entre las predicciones y las verdaderas calificaciones que le habían dado a los juegos.*

In [ ]:
df_svd = pd.read_csv('final_reviews.csv')
print(df_svd.shape)
df_svd.head()

In [ ]:
N_filas = 100000 # Limitamos el dataset a N_filas

data_svd = Dataset.load_from_df(df_svd[['username','product_id','rating']][:N_filas], reader)

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import KFold

# Se define un iterador de validación cruzada
kf = KFold(n_splits=3)

algo = SVD()

for trainset, testset in kf.split(data):

    # Algoritmo train and test.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Calcula e imprime el RMSE
    accuracy.rmse(predictions, verbose=True)

* *Usamos RMSE como evaluaciones de resultados de nuestras recomendaciones. A medida que se amplíe el conjunto de datos, se mejorarán las evaluaciones de RMSE.*

* *En éste caso, vemos que efectivamente el resultado obtenido fue mejor que el de KNN Basic.*

#### 2.2 OPTIMIZACIÓN DE HIPERPARÁMETROS

* *Realizaremos una optimización de hiperparámetros a partir de GridSearch, para hacer una búsqueda de fuerza bruta de los hiperparámetros para el algoritmo SVD.*

In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import GridSearchCV

param_grid = {'n_factors':[5,25,50], 'n_epochs': [5,10,20], 'lr_all': [0.001,0.002,0.005],
              'reg_all': [0.002, 0.02, 0.2]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)

gs.fit(data)

# Mejor RMSE Score
print(gs.best_score['rmse'])

# Combinación de parámetros que dan la mejor puntuación de RMSE
print(gs.best_params['rmse'])

* *Aquí estamos evaluando el RMSE promedio en un procedimiento de validación cruzada de 3 veces.*

* *Ahora podemos usar el algoritmo que produce el mejor RMSE.*
* *Tomamos la instancia del algoritmo con el conjunto óptimo de parámetros, y los utilizamos para realizar las predicciones:*

In [ ]:
algo_gs = gs.best_estimator['rmse']
algo_gs.fit(data.build_full_trainset())

In [ ]:
predictions_gs = algo_gs.test(testset)
accuracy.rmse(predictions_gs, verbose=True)

* *El valor del RMSE obtenido con el modelo SVD una vez aplicada la optimización de hiperparámetros, es realmente mejor que el obtenido del modelo Benchmark.*

#### 2.3 COMPARACION ENTRE MODELOS

* Comparación del desempeño de los modelos de ML utilizados con Surpr!se.

| Modelos con Surprise | RMSE |                   Hiperparámetros Utilizados                  |
|:--------------------:|:----:|:-------------------------------------------------------------:|
|       KNN Basic      | 1.24 |             neighbors min = 1, neighbors máx = 40             |
|          SVD         | 0.95 | n_factors = 5, n_epochs = 20, lr_all = 0.005, reg_all = 0.002 |

* *__La factorización Matricial con SVD__ fue el modelo con mejor desempeño, y mejor aún habiendo optimizado sus hiperparámetros.*
* *Éste resultado, era realmente el esperado, ya que se trata de un algoritmo eficiente y fácil de usar que ofrece un alto rendimiento y precisión en comparación con otros algoritmos con Surpr!se.*
* *Si bien los métodos de filtrado colaborativo basados en items (nuestro modelo Benchmark) o en usuarios son simples e intuitivos, las técnicas de factorización matricial suelen ser más efectivas porque nos permiten descubrir las características latentes que subyacen a las interacciones entre los usuarios y los elementos.*
* *Para ello, descomposición vectorial singular (SVD), emplea el uso del descenso de gradiente para minimizar el error al cuadrado entre la calificación predicha y la calificación real, obteniendo finalmente el mejor modelo.*

#### 2.4 REALIZANDO PREDICCIONES - RECOMENDACIÓN COLABORATIVA

*Exploraremos las característica de `predictions` y alguno de sus elementos.*

1. Realizamos **una predicción para un usuario en particular (aleatorio)**.

In [ ]:
predictions_gs[1]

# uid: Id del Usuario
# iid: Id del juego
# r_ui: calificación que le da a ese juego en particular (la conocida)
# est: estimación de la calificación (obtenida de SVD)
# was imposible = False: fue posible calificar el juego

* *En éste caso, la predicción parece haberse acercado bastante a la predicción efectivamente dada por el usuario uid.*

2. Realizamos **una predicción para un usuario y un juego determinados por nosotros** (mediante la función `predict`)

In [ ]:
algo_gs.predict(50262, 35143)

* *r_ui=None: El usuario no la calificó dicho juego.*

3. Observamos, cuáles son las **mejores predicciones realizadas y cuales las peores**.

In [ ]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

def invert_dictionary(dictionary):
    """Invert a dictionary
    Args: 
        dictionary (dict): A dictionary
    Returns:
        dict: inverted dictionary
    """
    return {v: k for k, v in dictionary.items()}

def surprise_trainset_to_df(trainset, col_user="uid", col_item="iid", col_rating="rating"): 
    df = pd.DataFrame(trainset.all_ratings(), columns=[col_user, col_item, col_rating])
    map_user = trainset._inner2raw_id_users if trainset._inner2raw_id_users is not None else invert_dictionary(trainset._raw2inner_id_users)
    map_item = trainset._inner2raw_id_items if trainset._inner2raw_id_items is not None else invert_dictionary(trainset._raw2inner_id_items)
    df[col_user] = df[col_user].map(map_user)
    df[col_item] = df[col_item].map(map_item)
    return df

trainset_df = surprise_trainset_to_df(trainset)

* *A través de Surpr!se, elaboramos un Dataframe con los usuarios películas y ratings.*

In [ ]:
trainset_df.head()

In [ ]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['Iu'] = df.uid.apply(get_Iu)
df['Ui'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)
best_predictions = df.sort_values(by='err')[:10]
worst_predictions = df.sort_values(by='err')[-10:]

* *Luego, podemos determinar el **error resultante** de las predicciones para combinación de usuario/juego, a partir de la diferencia existente entre la calificación real y la predicción realizada.*

In [ ]:
df.head()

* *Las ordenamos de mayor a menor y de menor a mayor.*

__MEJORES PREDICCIONES__

In [ ]:
best_predictions.head()

* *Se observan las mejores predicciones, donde el error es igual a 0.*

__PEORES PREDICCIONES__

In [ ]:
worst_predictions.head()

* *Obtenemos las peores predicciones, donde el error es cercano a 4, es decir que se calificó 1 y se predijo cerca de 5 o viceversa.*

4. Finalmente, visualiamos el **Top-N de recomendaciones para cada usuario de un conjunto de predicciones.**

In [ ]:
def get_top_n(predictions, n=5):

    # Primero mapea las predicciones a cada usuario.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Luego ordena las predicciones para cada usuario y trae las k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

* *Imprimimos los artículos recomendados para cada usuario.*

In [ ]:
from collections import defaultdict
top_n = get_top_n(predictions, n=5)

for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

* *Y las recomendaciones para un usuario en particular, con sus respectivos ratings.*

In [ ]:
print(top_n[538371])

5. Finalmente, vemos cuáles juegos le gustaron a un determinado usuario y **cuáles le recomienda el sistema**.

In [ ]:
usuario = 398896
rating = 4   # Le pedimos los juegos que tengan un rating de 4 o más
df_user = df_svd[(df_svd['username'] == usuario) & (df_svd['rating'] >= rating)]
df_user = df_user.reset_index(drop=True)
df_user['title'] = df_title['title'].loc[df_user.product_id].values
df_user

* *El juego que más le gustó al usuario fue Stranded Deep, seguido por Saints Row: The Third y Windward.*

* *Ahora, creamos el Dataframe donde vamos a guardar las recomendaciones para un usuario en particular.*

In [ ]:
recomendaciones_usuario = df_title.iloc[:398896].copy()
print(recomendaciones_usuario.shape)
recomendaciones_usuario.head()

* *Luego, quitamos del Dataframe todas los juegos que ya sabemos que vio.*

In [ ]:
usuario_vistas = df_svd[df_svd['username'] == usuario]
print(usuario_vistas.shape)
usuario_vistas

* *¡Vemos las recomendaciones que ya podemos hacerle a dicho usuario!*

In [ ]:
recomendaciones_usuario.drop(usuario_vistas.product_id, inplace = True)
recomendaciones_usuario = recomendaciones_usuario.reset_index()
recomendaciones_usuario.head()

* *__Y hacemos las recomendaciones!!!__, con el Id de juego específico, y ordenados de mayor a menor. Abajo la recomendación con su valor.*

In [ ]:
recomendaciones_usuario['Estimate_Score'] = recomendaciones_usuario['product_id'].apply(lambda x: algo.predict(usuario, x).est)

In [ ]:
recomendaciones_usuario = recomendaciones_usuario.sort_values('Estimate_Score', ascending=False)
print(recomendaciones_usuario.head(10))
# Recomendaciones con valuaciones estimadas por debajo

![FireworksUrl](https://seguridadpcs.files.wordpress.com/2018/12/fuegos-artificiales.gif?w=400 "fireworks")

## Parte C - Investigación

Con el fin de aplicar una herramienta más de las aprendidas en la seccción de 'Aplicaciones' de Acámica, ver su funcionamiento y resultados, sería interesante probar un **Sistema de recomendación basado en NLP (Natural Language Processing).**

La idea sería construir un Sistema de Recomendación Basado en Contenido con un enfoque de Procesamiento de Lenguaje Natural. 

¿En qué se diferencia de los sistemas de recomendación basados en contenido?
Los métodos de filtrado basados en contenido se basan en una descripción del elemento y un perfil de las preferencias del usuario. Estos métodos se adaptan mejor a situaciones en las que hay datos conocidos sobre un elemento (nombre, ubicación, descripción, etc.), pero no sobre el usuario. Los recomendadores basados en contenido tratan la recomendación como un problema de clasificación específico del usuario y aprenden un clasificador para los gustos y disgustos del usuario en función de las características del producto.

Si bien muchas veces basarse en contenidos no es lo ideal, ya que las preferencias de los usuarios cambian constantemente, éste enfoque es muy útil cuando se trata de productos que se destacan por una descripción o un título (datos de texto en general), como noticias, trabajos, libros, etc.  

... y en el **caso particular de Steam**, y como hemos idicado anteriormente, no contamos con un tipo de calificación específico y lo ideal sería poder transcribir y hacer uso de aquello que comentan los usuarios de cada juego a partir del feature `text`, donde puede estimarse qué es lo que se está pensando de esos juegos...

En relación a los **resultados esperados**, supongo se alcanzaría una predicción de juegos más razonable y **real**, ya que en el presente estudio, la determinación de los ratings por horas fue realizada ad hoc, sin profundizar en si la cantidad de horas jugadas de un juego en particular tienen relación con la antigüedad que tiene el juego, el género del mismo, si es gratis o no, si es de competición o no, etc.

Por medio de la librería __NTLK__, utilizada en los Notebooks de práctica y quizás otras más, esperaría poder realizar un buen Preprocesamiento de los Datos.  

Como guía, por ejemplo, podría servir el siguiente link encontrado de [Building NLP COntent Based Recommender Systems](https://medium.com/@armandj.olivares/building-nlp-content-based-recommender-systems-b104a709c042), aunque está claro que el Sistema de Recomendación a generar, requerirá mucha más lectura y estudio, como ocurrió con los proyectos hasta ahora confeccionados.